# Jotto

The best first guess is arets based on information gain.

- https://norvig.com/ngrams/

In [1]:
import lzma
from collections import Counter
import math
import pickle
from pathlib import Path
from functools import cache

from tqdm.auto import tqdm
import numpy as np

In [2]:
def load_word_list():
    with lzma.open('5-gram.txt.xz', mode='rt') as f:
        words = [line.strip() for line in f]
    return words

def compute_score(guess, secret):
    s = Counter(secret)
    g = Counter(guess)
    return sum([min(s.get(letter, 0), g.get(letter, 0)) for letter in g])

def load_scores(words):
    fn = 'jotto.npy'
    if Path(fn).exists():
        return np.load(fn)

    num_words = len(words)
    S = np.zeros((num_words, num_words), dtype=np.int32)
    for i, word1 in tqdm(enumerate(words), total=len(words)):
        for j, word2 in enumerate(words):
            S[j][i] = compute_score(word1, word2)
    np.save(fn, S)
    return load_scores(words)

def compute_entropy(probs):
    return -sum(p*math.log(p, 2) for p in probs)

def lookup_score(word1, word2):
    i = word2index[word1]
    j = word2index[word2]
    return S[j][i]

def compute_guess(words):
    best_word = None
    best_gain = 0
    N = len(words)
    if N == 1:
        return words[0]
    for guess in tqdm(words):
        counts = Counter([lookup_score(guess, w) for w in words])
        entropy = compute_entropy([v/N for v in counts.values()])
        if entropy > best_gain:
            best_gain = entropy
            best_word = guess
    return best_word

def apply_conditions(words, conditions):
    for guess, score in conditions.items():
        words = [w for w in words if lookup_score(w, guess) == score]
    return words

In [3]:
%%time
words = load_word_list()
num_words = len(words)
S = load_scores(words)
index2word = dict(enumerate(words))
word2index = {w: i for i, w in enumerate(words)}

CPU times: user 3.7 ms, sys: 111 ms, total: 115 ms
Wall time: 373 ms


In [4]:
%%time
words = load_word_list()
words = apply_conditions(words, {})
print('Best First Guess:', compute_guess(words))

  0%|          | 0/12597 [00:00<?, ?it/s]

Best First Guess: arets
CPU times: user 32.3 s, sys: 535 ms, total: 32.8 s
Wall time: 32.8 s


In [5]:
%%time
# Guess word
conditions = {
    'arets': 2,
}
words = load_word_list()
words = apply_conditions(words, conditions)
print(compute_guess(words))

  0%|          | 0/5041 [00:00<?, ?it/s]

eider
CPU times: user 5.36 s, sys: 72.8 ms, total: 5.44 s
Wall time: 5.43 s
